<a href="https://colab.research.google.com/github/Abhishek0697/Detection-of-Hate-Speech-in-Multimodal-Memes/blob/main/Code/Experiments/ResNeXt101_32x8d.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)



In [ ]:
!ls

drive  gdrive  sample_data


In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Kaggle"

In [ ]:
!kaggle datasets download -d parthplc/facebook-hateful-meme-dataset

facebook-hateful-meme-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
%cd /content/gdrive/MyDrive/Kaggle/facebook-hateful-meme-dataset

/content/gdrive/MyDrive/Kaggle/Facebook Hateful Meme Dataset


In [ ]:
!unzip \*.zip  && rm *.zip

In [ ]:
!pip install transformers


     |████████████████████████████████| 1.9MB 8.1MB/s 
     |████████████████████████████████| 3.2MB 37.3MB/s 
     |████████████████████████████████| 890kB 39.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=32b4da555d7d2e797e91ff99ed713ae5247a19fd8c4523ac378d3856e6e586e9
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
!nvidia-smi

Tue Mar  2 07:58:19 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
'''IMPORTING NECESSARY MODULES
'''
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset, TensorDataset
import sys
import torch.nn as nn
import torch.nn.functional as F
from torch.utils import data

import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter

sys.path.append('/content/gdrive/MyDrive/Kaggle/Hateful meme detection local files')

from dataloader import mydataset
from trainer_ResNet import train, test_classify
from resnet_models import ResNet,Bottleneck,resnext101_32x8d
from Load_model import load
from plot_curves import plot_loss, plot_acc

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


**Dataloading Scheme**

In [ ]:
trainlist = r'/content/gdrive/MyDrive/Kaggle/Facebook Hateful Meme Dataset/data/train.jsonl'
validlist = r'/content/gdrive/MyDrive/Kaggle/Facebook Hateful Meme Dataset/data/dev.jsonl'

In [ ]:
'''Train Dataloader''' 
train_dataset = mydataset(trainlist, name='train')          
train_dataloader = data.DataLoader(train_dataset, shuffle= True, batch_size = 28, num_workers=16,pin_memory=True)


'''Validation Dataloader''' 
validation_dataset = mydataset(validlist, name='valid')         
validation_dataloader = data.DataLoader(validation_dataset, shuffle=False, batch_size = 8, num_workers=16,pin_memory=True)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2155: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


**Model Definition**

Train Resnet50/ ResNext101

In [ ]:
modelname = 'ResneXt101_32x8d'
modelpath = '/content/'+modelname

In [ ]:
torch.cuda.is_available()

True

In [ ]:
# model = ResNet(Bottleneck, [3, 4, 6, 3], num_classes = 2)


'''
Pretrained ResNeXt101_32x8d
'''
model = resnext101_32x8d(pretrained = True)

model = nn.DataParallel(model).to(device)



# '''
# Load saved model from checkpoint
# '''
# model, optimizer, lr_scheduler, train_loss, v_loss, v_acc, epoch = load(modelpath, model, optimizer, lr_scheduler)

Downloading: "https://download.pytorch.org/models/resnext101_32x8d-8ba56ff5.pth" to /root/.cache/torch/hub/checkpoints/resnext101_32x8d-8ba56ff5.pth


In [ ]:
'''
Transfer Learning
'''

# for param in model.module.parameters():
#     param.requires_grad = False

fc_inputs = model.module.fc.in_features

model.module.fc = nn.Sequential(
    nn.Linear(fc_inputs, 2)
#     nn.BatchNorm1d(4096),
#     nn.ReLU(),
#     nn.Linear(768, 2)
    
)

model.to(device)

DataParallel(
  (module): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): S

**Hyperparameters**

In [ ]:
## Loss Function
criterion = nn.CrossEntropyLoss()

# Optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, weight_decay=1e-4, momentum=0.9)

lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size= 20, gamma = 0.1)

# Epochs
num_Epochs = 40


In [ ]:
writer = SummaryWriter("ResNeXt101_32x8d")

train(model, train_dataloader, validation_dataloader, criterion, optimizer, lr_scheduler, modelpath, writer, device, epochs = num_Epochs)\

writer.flush()
writer.close()

In [ ]:
'''
Load saved model from checkpoint
'''
model, optimizer, lr_scheduler, train_loss, v_loss, v_acc, epoch = load(modelpath, model, optimizer, lr_scheduler)

**Evaluate**

In [ ]:
test_classify(model, validation_dataloader, criterion, device)

In [ ]:
avg_loss = 0

for batch_num, (feats, captions, input_id, attention_masks, target) in enumerate(train_dataloader):
        feats, target = feats.to(device), target.to(device)


        '''
        Compute output and Loss
        '''
        output, embeddings = model(feats) 
        total_loss = criterion(output, target)
        
        
        avg_loss += total_loss.item()
        
        
avg_loss/len(train_dataloader)

In [ ]:
'''
Scale of embeddings
'''
embeddings[0][0:100]

In [ ]:
'''
Scale of output
'''
output[0]

In [ ]:
!cp "/content/ResNeXt101_32x8d" "/content/gdrive/MyDrive/Kaggle/saved_model_checkpoints"